- 이 파일 셀들을 복사해서 제출 notebook 만든다.
- 이 repo의 src, third-party, ckpt를 압축한다. 
  - ex) submission_files.zip
- 하나의 directory로 업로드 한다.
  - /kaggle/input/submission_files 로 올린다.
  - 압축 알아서 풀려있음

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import random
import torch
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
import argparse
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import sys
import pandas as pd
# 일단 __init__.py 추가해서 이렇게 해놨음.
BASE_PATH='/workspaces/kaggle-NFL-contact-detection'
sys.path.append(os.path.join(BASE_PATH, "src"))

# timm as third-party
sys.path.append(os.path.join(BASE_PATH, 'third-party/pytorch-image-models'))

from config import CFG
from factory.dataset_factory import DataSetFactory
from factory.lightning_module_factory import LightningModuleFactory
from train import seed_everything

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# TODO: data dir train/pred 구분.
CFG["dataset_params"]["data_dir"] = os.path.join(BASE_PATH, "data")
CFG["dataset_params"]["preprocess_result_dir"] = os.path.join(BASE_PATH, "data/processed")
CFG["is_prediction"] = True

In [4]:
seed_everything(CFG['seed'])
device_str = "cpu"
if torch.cuda.is_available():
    device_str = "cuda"
device_str

'cuda'

In [5]:
saved_model_path = os.path.join(
    BASE_PATH, 
    "epoch-9-step-29410-011619.ckpt"
)

In [6]:

model_name = f"{CFG['model_name']}-{CFG['model_version']}"
dataset_params = CFG['dataset_params']
model_params = CFG['model_params']
data_module = DataSetFactory.get_dataset(name=model_name,
                                            params=dataset_params)
lightning_module = LightningModuleFactory.get_lightning_module(name=model_name,
                                                                load_path=saved_model_path,
                                                                params=model_params)

trainer = pl.Trainer(max_epochs=CFG["epochs"],
                        accelerator=device_str,
                        devices=1 if device_str != "cpu" else None,
                        logger=False,
                        callbacks=[EarlyStopping(monitor="val_loss", mode="min", patience=3)])

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [7]:
predictions = trainer.predict(
    model=lightning_module, 
    datamodule=data_module)


====== [Preprocess] ======
- is_prediction: True
- run_type: test
------ [Loading Metadata] ------
------ [ffmpeg] ------
ffmpeg frames test


100%|██████████| 4/4 [00:00<00:00, 1314.11it/s]


------ [Mapping metadata] ------
Mapping video2helmets [size: 4]


100%|██████████| 4/4 [00:00<00:00, 400.52it/s]


-- Mapping video2frames: [size: 2]


100%|██████████| 2/2 [00:01<00:00,  1.41it/s]


------ [Preprocess helmet sensor data] ------
Expand contact id
Create features
------ [Setup dataset] ------
====== Generating dataset  ======
- stage: predict
------ [Load metadata] ------
-- Label count predict: 
contact
0    6631
Name: contact, dtype: int64
- predict_dataset_size: 6631


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 208/208 [01:22<00:00,  2.54it/s]


In [8]:
df_filtered = data_module.dataset_pred.df

In [9]:
y_pred = []
for p in predictions:
    y_pred += p.tolist() 
y_pred_np = np.array(y_pred)

In [10]:
len(y_pred_np), len(df_filtered)

(6631, 6631)

In [11]:
df_filtered['contact'] = (y_pred_np >= CFG["threshold"]).astype('int')
df_filtered.groupby("contact")["contact"].count()

contact
0    6093
1     538
Name: contact, dtype: int64

In [12]:

sub = pd.read_csv(os.path.join(CFG["dataset_params"]["data_dir"], 'sample_submission.csv'))

sub = sub.drop("contact", axis=1).merge(df_filtered[['contact_id', 'contact']], how='left', on='contact_id')
sub['contact'] = sub['contact'].fillna(0).astype('int')

sub[["contact_id", "contact"]].to_csv("submission.csv", index=False)

sub.head()

,contact_id,contact
0,58168_003392_0_38590_43854,0
1,58168_003392_0_38590_41257,0
2,58168_003392_0_38590_41944,0
3,58168_003392_0_38590_42386,0
4,58168_003392_0_38590_47944,0
